**CURSO**: *Machine Learning* en Geociencias<br />
**Profesor**: Edier Aristizábal (evaristizabalg@unal.edu.co) <br />
**Credits**: The content of this notebook is taken from several sources: Soner Yıldırım, Bex T, Manuel Hupperich, Youssef Hosni and Piero Paialunga en www.towardsdatascience.com. Every effort has been made to trace copyright holders of the materials used in this book. The author apologies for any unintentional omissions and would be pleased to add an acknowledgment in future editions.

# Time Series Analysis: LSTM

Long Short Term Memory or LSTM is a type of Neural Network, which was developed on the basis provided by the Recurrent Neural Networks, or RNN. The algorithms main usage falls into NLPs or Time Series, and the main idea behind it relates to processing the information they receive from the previous neuron and applying the activation function from scratch (as the CNN/RNN do), they actually divide the neuron into three main parts from which to set up the input from the next layer of neurons: Learn, Unlearn and Retain gate. The structure of the LSTM layer can be visualized in the image below:

![LSTM](https://miro.medium.com/max/700/0*RkFisWmsLWtGCByq.png)

#### Parameters for the LSTM
* PERCENTAGE = .98 #Split train/val and test set
* CALLBACK = .031 #Used to stop training the Network when the MAE from the validation set reached a perormance below 3.1%
* BATCH_SIZE = 20 #Number of samples that will be propagated through the network. I chose almost a trading month
* EPOCH = 50 #Settled to train the model
* WINDOW_LSTM = 30 #The window used for the input data
* PREDICTION_SCOPE = 0 #How many period to predict, being 0=1

In [12]:
import keras
import pandas as pd
import statsmodels.api as sm
from matplotlib import pyplot as plt
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense,Dropout
#from keras.optimizers import SGD

In [30]:
Q=pd.read_csv("https://github.com/edieraristizabal/MachineLearning/blob/master/data/CaudalHorario.csv", lineterminator='\r', error_bad_lines=False)
Q.head()

/tmp/ipykernel_28959/618929016.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  Q=pd.read_csv("https://github.com/edieraristizabal/MachineLearning/blob/master/data/CaudalHorario.csv", lineterminator='\r', error_bad_lines=False)


,"\n\n\n\n\n\n\n<!DOCTYPE html>\n<html lang=""en"" data-color-mode=""auto"" data-light-theme=""light"" data-dark-theme=""dark"" data-a11y-animated-images=""system"">\n <head>\n <meta charset=""utf-8"">\n <link rel=""dns-prefetch"" href=""https://github.githubassets.com"">\n <link rel=""dns-prefetch"" href=""https://avatars.githubusercontent.com"">\n <link rel=""dns-prefetch"" href=""https://github-cloud.s3.amazonaws.com"">\n <link rel=""dns-prefetch"" href=""https://user-images.githubusercontent.com/"">\n <link rel=""preconnect"" href=""https://github.githubassets.com"" crossorigin>\n <link rel=""preconnect"" href=""https://avatars.githubusercontent.com"">\n\n\n\n <link crossorigin=""anonymous"" media=""all"" rel=""stylesheet"" href=""https://github.githubassets.com/assets/light-5178aee0ee76.css"" /><link crossorigin=""anonymous"" media=""all"" rel=""stylesheet"" href=""https://github.githubassets.com/assets/dark-217d4f9c8e70.css"" /><link data-color-theme=""dark_dimmed"" crossorigin=""anonymous"" media=""all"" rel=""stylesheet"" data-href=""https://github.githubassets.com/assets/dark_dimmed-0adfa28f0e68.css"" /><link data-color-theme=""dark_high_contrast"" crossorigin=""anonymous"" media=""all"" rel=""stylesheet"" data-href=""https://github.githubassets.com/assets/dark_high_contrast-1c8575b36644.css"" /><link data-color-theme=""dark_colorblind"" crossorigin=""anonymous"" media=""all"" rel=""stylesheet"" data-href=""https://github.githubassets.com/assets/dark_colorblind-5113d2be20b0.css"" /><link data-color-theme=""light_colorblind"" crossorigin=""anonymous"" media=""all"" rel=""stylesheet"" data-href=""https://github.githubassets.com/assets/light_colorblind-c96add742484.css"" /><link data-color-theme=""light_high_contrast"" crossorigin=""anonymous"" media=""all"" rel=""stylesheet"" data-href=""https://github.githubassets.com/assets/light_high_contrast-290f92f5e867.css"" /><link data-color-theme=""light_tritanopia"" crossorigin=""anonymous"" media=""all"" rel=""stylesheet"" data-href=""https://github.githubassets.com/assets/light_tritanopia-cdd88f146bf7.css"" /><link data-color-theme=""dark_tritanopia"" crossorigin=""anonymous"" media=""all"" rel=""stylesheet"" data-href=""https://github.githubassets.com/assets/dark_tritanopia-2171ea0f078b.css"" />\n \n <link crossorigin=""anonymous"" media=""all"" rel=""stylesheet"" href=""https://github.githubassets.com/assets/primer-0e158a2f5ddd.css"" />\n <link crossorigin=""anonymous"" media=""all"" rel=""stylesheet"" href=""https://github.githubassets.com/assets/global-bec0587a3deb.css"" />\n <link crossorigin=""anonymous"" media=""all"" rel=""stylesheet"" href=""https://github.githubassets.com/assets/github-fb643df33dec.css"" />\n <link crossorigin=""anonymous"" media=""all"" rel=""stylesheet"" href=""https://github.githubassets.com/assets/code-3d7b701fc6eb.css"" />\n\n <meta name=""optimizely-datafile"" content=""{&quot;groups&quot;: []",&quot;environmentKey&quot;: &quot;production&quot;,&quot;rollouts&quot;: [],&quot;typedAudiences&quot;: [],&quot;projectId&quot;: &quot;16737760170&quot;,&quot;variables&quot;: [],&quot;featureFlags&quot;: [],&quot;experiments&quot;: [{&quot;status&quot;: &quot;Running&quot;,&quot;audienceIds&quot;: [],&quot;variations&quot;: [{&quot;variables&quot;: [],...,&quot;action&quot;:&quot;go to Pricing&quot;,"&quot;label&quot;:&quot;text:Pricing&quot;}"">Pricing</a></li>\n <li class=""mr-3 mr-lg-0""><a href=""https://docs.github.com"" data-analytics-event=""{&quot;category&quot;:&quot;Footer&quot;",&quot;action&quot;:&quot;go to api&quot;,"&quot;label&quot;:&quot;text:api&quot;}"">API</a></li>\n <li class=""mr-3 mr-lg-0""><a href=""https://services.github.com"" data-analytics-event=""{&quot;category&quot;:&quot;Footer&quot;",&quot;action&quot;:&quot;go to training&quot;,"&quot;label&quot;:&quot;text:training&quot;}"">Training</a></li>\n <li class=""mr-3 mr-lg-0""><a href=""https://github.blog"" data-analytics-event=""{&quot;category&quot;:&quot;Footer&quot;",